# Tutorial4: visualize recovery results

---
This tutorial demonstrates visualize the 3D / 4D recovery results. \
By sampling a trained neural network at regular grid intervals we can visualize the recovered 3D emission.

In [1]:
import bhnerf
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
from jax import numpy as jnp
from flax.training import checkpoints
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-u0owrhje because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-01-26 15:54:36.086507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [2]:
"""
Define network architecture and load latest checkpoint from directory
Change the path of checkpoint_dir to where the checkpoint is saved.
"""
checkpoint_dir = 'checkpoints/sparsity_new/full_image.nspots5.true_axis.2022-01-20.10:36:30/'

orbit_args = {'tstart': 0.0, 'tstop': 1.0, 'axis_init': jnp.array([-0.4556, 0.5549, 0.6961]), 'velocity': 1.0 }
predictor = bhnerf.network.NeRF3D_RotationAxis()
emission_op = bhnerf.network.EmissionOperator(predictor, orbit_args)
state = checkpoints.restore_checkpoint(checkpoint_dir, None)

In [3]:
"""
Define a VolumeVisualizer object
"""
resolution = 128
visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

In [8]:
"""
Define the position of the camera and sample/integrate emission according to the camera rays.
View the estimated emission at t=0 from multiple angles. 
Note that jit is useful for acceleration however the initial rendering will be slow due to run time compilation.
"""
images = []
for azimuth in tqdm(np.linspace(0.0, 360, 6)):
    visualizer.set_view(radius=27.5, azimuth=azimuth, zenith=np.pi/3)
    t, x, y, z = jnp.ones_like(visualizer.x), visualizer.x, visualizer.y, visualizer.z
    emission = emission_op(state['params'], x, y, z, t)
    emission = bhnerf.emission.zero_unsupervised_emission(emission, x, y, z, rmin=2.0, rmax=5.0)
    images.append(visualizer.render(emission, jit=True, norm_const=1.0, bh_radius=2.0))

  0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
%matplotlib widget
fig, axes = plt.subplots(1, 6, figsize=(10,2))
for ax, image in zip(axes, images):
    ax.imshow(image)
    ax.axis('off')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
"""
View the estimated emission at from the same angle at different times using the estimated rotation axis.
"""
images = []
for frame in tqdm(np.linspace(0.0, 1.0, 6)):
    visualizer.set_view(radius=27.5, azimuth=np.pi/4, zenith=np.pi/4)
    t, x, y, z = frame * jnp.ones_like(visualizer.x), visualizer.x, visualizer.y, visualizer.z
    emission = emission_op(state['params'], x, y, z, t)
    emission = bhnerf.emission.zero_unsupervised_emission(emission, x, y, z, rmin=2.0, rmax=5.0)
    images.append(visualizer.render(emission, jit=True, norm_const=1.0, bh_radius=2.0))

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
%matplotlib widget
fig, axes = plt.subplots(1, 6, figsize=(10,2))
for ax, image in zip(axes, images):
    ax.imshow(image)
    ax.axis('off')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …